In [2]:
# !pip install transformers
# !pip install emoji
# !pip install pyspellchecker
# !pip install pandas
# !pip install matplotlib tqdm nltk
# !pip install scikit-learn

In [51]:
import torch
import gc

torch.cuda.empty_cache()

gc.collect()
torch.cuda.synchronize()


In [4]:
import numpy as np
import pandas as pd
import string
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm

import re

import emoji

from spellchecker import SpellChecker
spell = SpellChecker()

import nltk
nltk.download("punkt")
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

import random
from sklearn import preprocessing

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
# Define a function to correct spelling mistakes in a string
def correct_spelling(text):
    words = text.split()
    corrected_words = []
    for word in words:
        corrected_word = spell.correction(word)
        if corrected_word != None:
          corrected_words.append(corrected_word)
        # print(corrected_word)
    corrected_text = ' '.join(corrected_words)
    return corrected_text

In [6]:
abbreviations = {
    "w/": "with",
    "w/o": "without",
    "msg": "message",
    "u": "you",
    "r": "are",
    "lol": "laughing out loud",
    "np": "no problem",
    "LOL": "laughing out loud",
    "XD" : "laugh",
    "xd" : "laugh"
}

# Define a function to correct abbreviations in a string
def correct_abbreviations(text, abbreviations):
    words = text.split()
    corrected_words = []
    for word in words:
        if word in abbreviations:
            corrected_word = abbreviations[word]
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)
    corrected_text = ' '.join(corrected_words)
    return corrected_text

In [15]:
import pandas as pd

df = pd.read_csv('TweetsCOV19_preprocessed.csv', encoding='utf-8')

In [16]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10472 entries, 0 to 10471
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Sentiment     10472 non-null  object
 1   TweetText     10462 non-null  object
 2   tweet_length  10472 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 245.6+ KB


In [17]:
# way too fucking slow
tqdm.pandas()


df_new = df[:]
# df_new["tweet"] = df_new["TweetText"].progress_apply(preprocess_tweets)
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10472 entries, 0 to 10471
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Sentiment     10472 non-null  object
 1   TweetText     10462 non-null  object
 2   tweet_length  10472 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 245.6+ KB


In [18]:
def make_dataset(sentiment):
    try:
        y = np.fromstring(sentiment, dtype=int, sep=' ')[:2]
        return y
    except:
        return None


In [20]:
df_intermediate = pd.DataFrame()
df_intermediate['x'] = df_new['TweetText'].astype('<U1').to_numpy() # string
df_intermediate['Y'] = df_new['Sentiment'].apply(make_dataset).to_numpy()

# Drop non-values or non-NumPy arrays from Y column
df_intermediate = df_intermediate.dropna()
df_intermediate = df_intermediate[df_intermediate['Y'].apply(lambda x: isinstance(x, np.ndarray))]


# Split Y column into Y_pos and Y_neg columns
df_intermediate[['Y_pos', 'Y_neg']] = pd.DataFrame(df_intermediate['Y'].tolist())

# Remove the original Y column
df_intermediate.drop('Y', axis=1, inplace=True)
df_intermediate = df_intermediate.dropna()

print(df_intermediate)


                                                       x  Y_pos  Y_neg
0      hey re go back listen rip hung cry jump proud ...      3     -4
1      arrive rainstorm appear little slower thursday...      1     -1
2      little munchkin little fan show stopper today ...      1     -3
3      already cancel publicly time got uncouth boor ...      2     -4
4      hardware store maybe nicole hardware appear so...      1     -1
...                                                  ...    ...    ...
10467  look julian one peopl share grievance onto mas...      1     -4
10468  mi african rapper best rapper africa mean they...      1     -1
10469  serv fellow man said definite man ist unwrap b...      1     -3
10470  skee ball retail appear pinball barca tonight ...      3     -1
10471        sorry brother happen youfeel respect : : to      3     -2

[10472 rows x 3 columns]


In [21]:
df_intermediate

,x,Y_pos,Y_neg
0,hey re go back listen rip hung cry jump proud ...,3,-4
1,arrive rainstorm appear little slower thursday...,1,-1
2,little munchkin little fan show stopper today ...,1,-3
3,already cancel publicly time got uncouth boor ...,2,-4
4,hardware store maybe nicole hardware appear so...,1,-1
...,...,...,...
10467,look julian one peopl share grievance onto mas...,1,-4
10468,mi african rapper best rapper africa mean they...,1,-1
10469,serv fellow man said definite man ist unwrap b...,1,-3
10470,skee ball retail appear pinball barca tonight ...,3,-1


In [22]:
x,y1,y2 = df_intermediate['x'], df_intermediate['Y_pos'], df_intermediate['Y_neg']

In [23]:
import pandas as pd

# Assuming df_intermediate is your dataframe

# Create one-hot encodings for y1
y1_one_hot = pd.get_dummies(df_intermediate['Y_pos'], prefix='Y_pos', dtype=int).to_numpy()

# Create one-hot encodings for y2
y2_one_hot = pd.get_dummies(df_intermediate['Y_neg'], prefix='Y_neg', dtype=int).to_numpy()

# Concatenate the one-hot encoded columns with the original dataframe
# df_encoded = pd.concat([df_intermediate['x'], y1_one_hot, y2_one_hot], axis=1)
# df_encoded
y1_one_hot, y2_one_hot

(array([[0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        ...,
        [1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0]]),
 array([[0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1],
        [0, 0, 1, 0, 0],
        ...,
        [0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0]]))

In [24]:
import torch

# Horizontally stack y1 and y2 into Y
Y = np.concatenate([y1_one_hot, y2_one_hot], axis=1)

Y = Y.reshape(-1, 2, 5)
Y

array([[[0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0]],

       [[1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1]],

       [[1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0]],

       ...,

       [[1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0]],

       [[0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1]],

       [[0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0]]])

## Train the model

In [84]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast, BertModel, AutoTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


# Load model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained(model_name)
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=y_train[:,0].shape[1])
# model.softmax_f = nn.Softmax(dim=1)

# two separate classifiers
model.classifier1 = nn.Sequential(
    nn.Linear(model.config.hidden_size, 16),
    nn.ReLU(),
    nn.Linear(16, 5),
    nn.Softmax(dim=1)
)

# # Define the second classifier with softmax activation
# model.classifier2 = nn.Sequential(
#     nn.Linear(model.config.hidden_size, 16),
#     nn.ReLU(),
#     nn.Linear(16, 5),
#     nn.Softmax(dim=1)
# )


# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Split dataset
x_train, x_test, y_train, y_test = train_test_split(x, Y, test_size=0.2, random_state=42)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [59]:
# Convert dataset to PyTorch tensors
train_encodings = tokenizer(list(x_train), truncation=True, padding=True, return_tensors="pt")
test_encodings = tokenizer(list(x_test), truncation=True, padding=True, return_tensors="pt")

In [60]:
train_encodings[0]

Encoding(num_tokens=158, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [85]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels1, labels2):
        self.encodings = encodings
        self.labels1 = labels1
        self.labels2 = labels2

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels1'] = torch.tensor(self.labels1[idx], dtype=torch.float)
        item['labels2'] = torch.tensor(self.labels2[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels1)


In [86]:
# Create datasets
train_dataset = SentimentDataset(train_encodings, y_train[:,0], y_train[:,1])
test_dataset = SentimentDataset(test_encodings, y_test[:,0], y_test[:,1])

# Define dataloaders
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=16)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=64)

# Define loss function, optimizer, and learning rate scheduler
loss_function = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-5)#, weight_decay=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)


In [72]:
for item in train_dataloader:
    print(item['input_ids'].shape)
    break

torch.Size([16, 158])


/tmp/ipykernel_155/1923553546.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [73]:
# del input_ids,attention_mask,labels1,labels2,logits1,logits2
torch.cuda.empty_cache()
optimizer.zero_grad()

In [93]:
import logging

# Configure logging
logging.basicConfig(filename='training.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


num_epochs = 30
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    model.train()
    total_train_loss_1 = 0
    # total_train_loss_2 = 0
    batch_id = 0
    for batch in tqdm(train_dataloader):
        batch_id += 1
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        # input_ids = input_ids.float()
        attention_mask = batch['attention_mask'].to(device)
        
        labels1 = batch['labels1'].to(device)
        # labels2 = batch['labels2'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask)#.float()
        logits1 = model.classifier1(outputs.last_hidden_state[:, 0, :])
        # logits1 = model.classifier1(outputs.last_hidden_state[:, 0, :])
        # logits2 = model.classifier2(outputs.last_hidden_state[:, 0, :])
              
        loss1 = loss_function(logits1, labels1)
        # loss2 = loss_function(logits2, labels2)
        
        total_train_loss_1 += loss1.item()
        # total_train_loss_2 += loss2.item()
        
        loss1.backward()
        # loss2.backward()
        
        optimizer.step()
        
        del input_ids,attention_mask,labels1, logits1#,labels2,logits1,logits2
        torch.cuda.empty_cache()
        
        
        # if batch_id % 150 == 0:
           
            # Evaluation
    model.eval()
    total_eval_loss_1 = 0
    # total_eval_loss_2 = 0
    test_batch_num = 0
    for batch in test_dataloader:
        test_batch_num += 1
        if test_batch_num > 25:
            break

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels1 = batch['labels1'].to(device)
        # labels2 = batch['labels2'].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            logits1 = model.classifier1(outputs.last_hidden_state[:, 0, :])
            # logits1 = model.classifier1(outputs.last_hidden_state[:, 0, :])
            # logits2 = model.classifier2(outputs.last_hidden_state[:, 0, :])

            loss1 = loss_function(logits1, labels1)
            # loss2 = loss_function(logits2, labels2)

        total_eval_loss_1 += loss1.item()
        # total_eval_loss_2 += loss2.item()

        del input_ids,attention_mask,labels1,logits1,#labels2,logits1,logits2

    avg_train_loss_1 = total_train_loss_1 / test_batch_num
    # avg_train_loss_2 = total_train_loss_2 / test_batch_num
    avg_eval_loss_1 = total_eval_loss_1 / test_batch_num
    # avg_eval_loss_2 = total_eval_loss_2 / test_batch_num

    # print(f"Average training loss (Y_pos): {avg_train_loss_1:.2f}")
    # print(f"Average training loss (Y_neg): {avg_train_loss_2:.2f}")
    # print(f"Average evaluation loss (Y_pos): {avg_eval_loss_1:.2f}")
    # print(f"Average evaluation loss (Y_neg): {avg_eval_loss_2:.2f}")c

    logging.info(f"BATCH stats for {batch_id}")
    logging.info(f"Average training loss (Y_pos): {avg_train_loss_1:.2f}")
    # logging.info(f"Average training loss (Y_neg): {avg_train_loss_2:.2f}")
    logging.info(f"Average evaluation loss (Y_pos): {avg_eval_loss_1:.2f}")
    # logging.info(f"Average evaluation loss (Y_neg): {avg_eval_loss_2:.2f}")
            
    if epoch % 1 == 0:
        torch.save(model.state_dict(), f"model_saved.pt")

    
    # Update learning rate
    scheduler.step()
    



Epoch 1/30


  0%|          | 0/524 [00:00<?, ?it/s]

/tmp/ipykernel_155/1923553546.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 2/30


  0%|          | 0/524 [00:00<?, ?it/s]

...

Epoch 26/30


  0%|          | 0/524 [00:00<?, ?it/s]

Epoch 27/30


  0%|          | 0/524 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [94]:
from sklearn.metrics import accuracy_score

model.eval()
total_eval_loss_1 = 0
# total_eval_loss_2 = 0
test_batch_num = 0

true_labels1 = []
pred_labels1 = []
# true_labels2 = []
# pred_labels2 = []

for batch in test_dataloader:
    test_batch_num += 1
    if test_batch_num > 1:
        break

    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels1 = batch['labels1'].to(device)
    # labels2 = batch['labels2'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits1 = model.classifier1(outputs.last_hidden_state[:, 0, :])
        
        # loss1 = loss_function(logits1, labels1)
        
        label = torch.argmax(labels1, dim=1)
        # prediction = torch.argmax(logits1)
        

        true_labels1.extend(label.cpu().numpy())
        pred_labels1.extend(torch.argmax(logits1, dim=1).cpu().numpy())

    # total_eval_loss_1 += loss1.item()

    del input_ids, attention_mask, labels1, logits1

avg_train_loss_1 = total_train_loss_1 / test_batch_num
avg_eval_loss_1 = total_eval_loss_1 / test_batch_num

accuracy1 = accuracy_score(true_labels1, pred_labels1)

# print(f"Average training loss (Y_pos): {avg_train_loss_1:.2f}")
# print(f"Average evaluation loss (Y_pos): {avg_eval_loss_1:.2f}")
print(f"Accuracy (Y_pos): {accuracy1:.2f}")


Accuracy (Y_pos): 0.39


/tmp/ipykernel_155/1923553546.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
# get cuda allocation info or sth

import torch
import gc

# Perform operations that use CUDA memory

# Get all objects in memory
objects = gc.get_objects()

# Filter objects that are tensors or models and print their CUDA memory usage
for obj in objects:
    if isinstance(obj, torch.Tensor) or isinstance(obj, torch.nn.Module):
        memory_usage = obj.numel() * obj.element_size()
        if obj.is_cuda:
            print(f"{type(obj).__name__}: {memory_usage / 1024**2:.2f} MB")
